In [ ]:
!pip install spacy
!pip install torchvision 
!pip install torchtext

In [1]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import spacy

from tqdm import tqdm

import torch
from torchtext.legacy import data
import torch.optim as optim

In [2]:
TEXT = data.Field(tokenize = 'spacy',
                  tokenizer_language = 'en_core_web_sm',
                  include_lengths = True)

LABEL = data.LabelField(dtype = torch.float)

In [3]:
fields_train = [('text', TEXT), ('label', LABEL)]
train_data = data.TabularDataset(path = 'products_sentiment_train.tsv',
                                        format = 'tsv',
                                        fields = fields_train,
                                        skip_header = True)

In [4]:
fields_test = [(None, None), ('text', TEXT)]
test_data = data.TabularDataset(path = 'products_sentiment_test.tsv',
                                        format = 'tsv',
                                        fields = fields_test,
                                        skip_header = True)

In [5]:
train_data, valid_data = train_data.split(random_state = np.random.seed())

In [6]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

In [7]:
BATCH_SIZE = 64

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    sort_key = lambda x: len(x.text),
    sort_within_batch=True,
    batch_size = BATCH_SIZE,
    device = device)

In [61]:
import torch.nn as nn

class RNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, output_dim, n_layers, 
                 bidirectional, dropout, pad_idx):
        
        super().__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        
        self.rnn = nn.LSTM(embedding_dim, 
                           hidden_dim, 
                           num_layers=n_layers, 
                           bidirectional=bidirectional, 
                           dropout=dropout)
        
        self.fc = nn.Linear(hidden_dim * 2, output_dim)
        
        self.dropout = nn.Dropout(dropout)

    def forward(self, text, text_lengths):
        
        embedded = self.dropout(self.embedding(text))
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, text_lengths.to('cpu'))
        packed_output, (hidden, cell) = self.rnn(packed_embedded)
        output, output_lengths = nn.utils.rnn.pad_packed_sequence(packed_output)
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
            
        return self.fc(hidden)

In [34]:
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
HIDDEN_DIM = 256
OUTPUT_DIM = 1
N_LAYERS = 2
BIDIRECTIONAL = True
DROPOUT = 0.5
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

model = RNN(INPUT_DIM, 
            EMBEDDING_DIM, 
            HIDDEN_DIM, 
            OUTPUT_DIM, 
            N_LAYERS, 
            BIDIRECTIONAL, 
            DROPOUT, 
            PAD_IDX)

In [35]:
pretrained_embeddings = TEXT.vocab.vectors
model.embedding.weight.data.copy_(pretrained_embeddings)

tensor([[-0.5636, -0.6551,  2.9162,  ...,  1.4803,  0.0057,  1.1102],
        [-0.8177, -0.5299,  1.5776,  ...,  1.0584,  1.8107, -2.1398],
        [-0.0382, -0.2449,  0.7281,  ..., -0.1459,  0.8278,  0.2706],
        ...,
        [-0.0205, -0.4512,  0.6993,  ...,  0.0381,  0.2786,  0.2889],
        [-0.3916,  0.0405, -0.3635,  ..., -0.0546, -0.0856, -0.0657],
        [-0.0540,  0.0612,  0.5509,  ..., -0.2635, -0.3830, -1.6478]])

In [36]:
UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

In [37]:
optimizer = optim.Adam(model.parameters())
criterion = nn.BCEWithLogitsLoss()

model = model.to(device)
criterion = criterion.to(device)

In [39]:
def binary_accuracy(preds, y):
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division 
    acc = correct.sum() / len(correct)
    return acc

In [40]:
def train(model, iterator, optimizer, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.train()
    
    for batch in iterator:
        
        optimizer.zero_grad()
        
        text, text_lengths = batch.text
        
        predictions = model(text, text_lengths).squeeze(1)
        
        loss = criterion(predictions, batch.label)
        
        acc = binary_accuracy(predictions, batch.label)
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [41]:
def evaluate(model, iterator, criterion):
    
    epoch_loss = 0
    epoch_acc = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:

            text, text_lengths = batch.text
            
            predictions = model(text, text_lengths).squeeze(1)
            
            loss = criterion(predictions, batch.label)
            
            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()
        
    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [42]:
from tqdm import tqdm
N_EPOCHS = 15

best_valid_loss = float('inf')

for epoch in tqdm(range(N_EPOCHS)):

    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    valid_loss, valid_acc = evaluate(model, valid_iterator, criterion)
     
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}%')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc*100:.2f}%')

  7%|▋         | 1/15 [00:01<00:14,  1.03s/it]

	Train Loss: 0.653 | Train Acc: 63.84%
	 Val. Loss: 0.661 |  Val. Acc: 59.58%


 13%|█▎        | 2/15 [00:01<00:12,  1.06it/s]

	Train Loss: 0.614 | Train Acc: 65.97%
	 Val. Loss: 0.693 |  Val. Acc: 52.76%


 20%|██        | 3/15 [00:02<00:11,  1.08it/s]

...

	Train Loss: 0.255 | Train Acc: 89.68%
	 Val. Loss: 0.541 |  Val. Acc: 77.71%


 93%|█████████▎| 14/15 [00:12<00:00,  1.13it/s]

	Train Loss: 0.193 | Train Acc: 92.22%
	 Val. Loss: 0.545 |  Val. Acc: 77.76%


100%|██████████| 15/15 [00:13<00:00,  1.11it/s]

	Train Loss: 0.193 | Train Acc: 91.18%
	 Val. Loss: 0.638 |  Val. Acc: 75.31%


In [43]:
model.load_state_dict(torch.load('tut2-model.pt'))

<All keys matched successfully>

In [44]:
nlp = spacy.load('en_core_web_sm')

def predict_sentiment(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, length_tensor))
    return prediction.item()

In [45]:
 predict_sentiment(model, "good amazing wow")

0.0065348828211426735